In [33]:
import requests
import base64
import pandas as pd
import re
from supabase import create_client
import os
from datetime import datetime
from dotenv import load_dotenv

# for job page scraper
from bs4 import BeautifulSoup
import openai
from time import sleep
import json

load_dotenv()


True

#### Make GET request to job board README

In [34]:
def get_github_readme():
    url = "https://api.github.com/repos/SimplifyJobs/Summer2025-Internships/contents/README.md"
    response = requests.get(url)
    
    if response.status_code == 200:
        content = response.json()['content']
        decoded_content = base64.b64decode(content).decode('utf-8')
        return decoded_content
    else:
        return f"Error: {response.status_code}"

In [35]:
def extract_url_from_html(html_string):
    # Extract URL from href attribute
    url_match = re.search(r'href="([^"]+)"', html_string)
    if url_match:
        return url_match.group(1)
    return html_string

#### Extract table data from HTML

In [45]:
def extract_table_data(markdown_content):
    # Find the table in the markdown content
    table_pattern = r'\| Company \| Role \| Location \| Application\/Link \| Date Posted \|\n\|[^\n]+\n((?:\|[^\n]+\n)*)'
    match = re.search(table_pattern, markdown_content)
    
    if match:
        table_content = match.group(0)
        
        # Convert markdown table to list of lists
        rows = table_content.split('\n')
        # Remove empty rows and the separator row (|----|)
        rows = [row for row in rows if row.strip() and not row.strip().startswith('|-')]
        
        # Parse each row
        data = []
        for row in rows[1:]:  # Skip header row
            # Split by | and remove empty strings
            cols = [col.strip() for col in row.split('|') if col.strip()]
            
            # Skip rows that don't have enough columns or contain only dashes
            if len(cols) < 4 or all(c.replace('-', '').strip() == '' for c in cols):
                continue
                
            # Clean up the company name (extract just the name from markdown link)
            company_raw = cols[0]
            # Extract company name from markdown link if present, otherwise use as is
            company_match = re.search(r'\[([^\]]+)\]', company_raw)
            company = company_match.group(1) if company_match else company_raw.replace('*', '')
            
            # Extract URL from HTML link
            application_link = extract_url_from_html(cols[3])
            
            # Create row data with safe indexing
            data.append({
                'Company': company,
                'Role': cols[1] if len(cols) > 1 else '',
                'Location': cols[2] if len(cols) > 2 else '',
                'Application_Link': application_link,
                'Date_Posted': cols[4] if len(cols) > 4 else ''
            })
        
        # Create DataFrame
        df = pd.DataFrame(data)
        return df
    
    return None

def extract_table_data_today(markdown_content):
    # Find the table in the markdown content
    table_pattern = r'\| Company \| Role \| Location \| Application\/Link \| Date Posted \|\n\|[^\n]+\n((?:\|[^\n]+\n)*)'
    match = re.search(table_pattern, markdown_content)
    
    if match:
        table_content = match.group(0)
        
        # Convert markdown table to list of lists
        rows = table_content.split('\n')
        # Remove empty rows and the separator row (|----|)
        rows = [row for row in rows if row.strip() and not row.strip().startswith('|-')]
        
        # Parse each row
        data = []
        for row in rows[1:]:  # Skip header row
            # Split by | and remove empty strings
            cols = [col.strip() for col in row.split('|') if col.strip()]
            
            # Skip rows that don't have enough columns or contain only dashes
            if len(cols) < 4 or all(c.replace('-', '').strip() == '' for c in cols):
                continue
                
            # Clean up the company name (remove ** if present)
            company = cols[0].replace('*', '')
            
            # Extract URL from HTML link
            application_link = extract_url_from_html(cols[3])
            
            # Create row data with safe indexing
            data.append({
                'Company': company,
                'Role': cols[1] if len(cols) > 1 else '',
                'Location': cols[2] if len(cols) > 2 else '',
                'Application_Link': application_link,
                'Date_Posted': cols[4] if len(cols) > 4 else ''
            })
        
        # Create DataFrame
        df = pd.DataFrame(data)
        
        # Filter for today's date
        today = datetime.now().strftime('%b %d')
        df = df[df['Date_Posted'] == today]
        
        return df
    
    return None

#### Open job page website, scrape text, and run through GPT to output job details

In [53]:
def enrich_job_data(df, max_workers=10):
    """
    Enrich job data using multithreading to process multiple jobs concurrently.
    
    Args:
        df: DataFrame containing job listings
        max_workers: Maximum number of concurrent threads (default: 10)
    """
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import warnings
    warnings.filterwarnings('ignore')
    
    # Create a copy of the DataFrame
    df = df.copy()
    
    # Initialize OpenAI client
    client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    
    # Initialize columns
    df['company_formatted'] = ''
    df['description'] = ''
    df['soft_skills'] = ''
    df['technical_skills'] = ''
    df['experience_level'] = ''
    
    def process_single_job(row_data):
        """Process a single job listing"""
        index, row = row_data
        
        try:
            # Skip rows with "↳" as company name
            if row['Company'].strip() == "↳":
                return index, {
                    'status': 'skipped',
                    'message': 'Duplicate entry',
                    'data': None
                }
                
            # Skip if application link contains 🔒
            if "🔒" in str(row['Application_Link']):
                return index, {
                    'status': 'skipped',
                    'message': 'Locked job posting',
                    'data': None
                }

            # Headers to mimic a browser request
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }

            # Get the webpage content
            response = requests.get(
                row['Application_Link'],
                headers=headers,
                timeout=10,
                verify=False
            )
            
            if response.status_code != 200:
                return index, {
                    'status': 'error',
                    'message': f'Failed to fetch webpage: Status code {response.status_code}',
                    'data': None
                }

            # Parse the webpage content
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Remove scripts and styles
            for script in soup(["script", "style"]):
                script.decompose()
            
            # Extract and clean text
            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = ' '.join(chunk for chunk in chunks if chunk)
            
            if not text.strip():
                return index, {
                    'status': 'error',
                    'message': 'No text content found',
                    'data': None
                }

            # Prepare prompt for GPT
            prompt = f"""
            Please analyze this job posting and extract the following information in JSON format:

            Job Description: {text[:4000]}

            Return a valid JSON object with exactly these fields:
            {{
                "company_formatted": "The name of the company",
                "description": "A concise summary of the job posting",
                "soft_skills": ["skill1", "skill2", ...],
                "technical_skills": ["skill1", "skill2", ...],
                "experience_level": "one of: beginner, intermediate, advanced"
            }}
            """
            
            # Make OpenAI API call
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that analyzes job postings and extracts key information in strict JSON format."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3
            )
            
            # Process the response
            response_content = response.choices[0].message.content.strip()
            
            # Remove code fences if present
            if response_content.startswith("```") and response_content.endswith("```"):
                response_content = response_content[3:]
                if '\n' in response_content:
                    first_line, rest = response_content.split('\n', 1)
                    if re.match(r'^\w+$', first_line.strip()):
                        response_content = rest
                if response_content.endswith("```"):
                    response_content = response_content[:-3]
            response_content = response_content.strip()
            
            # Parse JSON response
            parsed_data = json.loads(response_content)
            
            return index, {
                'status': 'success',
                'message': 'Successfully processed',
                'data': parsed_data
            }

        except Exception as e:
            return index, {
                'status': 'error',
                'message': str(e),
                'data': None
            }

    # Process jobs in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all jobs
        future_to_job = {
            executor.submit(process_single_job, (index, row)): (index, row) 
            for index, row in df.iterrows()
        }
        
        # Process completed jobs
        for future in as_completed(future_to_job):
            index, result = future.result()
            
            # Print progress
            print(f"\nProcessing job {index + 1}/{len(df)}: {df.iloc[index]['Company']} - {df.iloc[index]['Role']}")
            print(f"Status: {result['status']} - {result['message']}")
            
            # Update DataFrame if processing was successful
            if result['status'] == 'success' and result['data']:
                parsed_data = result['data']
                df.loc[index, 'company_formatted'] = parsed_data.get('company_formatted', '')
                df.loc[index, 'description'] = parsed_data.get('description', '')
                df.loc[index, 'soft_skills'] = ', '.join(parsed_data.get('soft_skills', []))
                df.loc[index, 'technical_skills'] = ', '.join(parsed_data.get('technical_skills', []))
                df.loc[index, 'experience_level'] = parsed_data.get('experience_level', '')
            
            # Add small delay to respect rate limits
            sleep(0.1)
    
    return df

In [54]:
readme_content = get_github_readme()
df = extract_table_data(readme_content)

df.head()

,Company,Role,Location,Application_Link,Date_Posted
0,Zoox,Developer Platforms Intern,"Foster City, CA",https://jobs.lever.co/zoox/3021c042-c2e9-495f-...,Dec 15
1,Astranis Space Technologies,DevOps Engineer (Flight Software) - Intern 🇺🇸,"San Francisco, CA",https://job-boards.greenhouse.io/astranis/jobs...,Dec 15
2,ABB,Data Science Intern- Summer 2025,"Cary, NC",https://careers.abb/global/en/job/ABB1GLOBAL94...,Dec 15
3,Staples,Software Engineering Intern (Quill - Hybrid),"Lincolnshire, IL",https://careers.staples.com/en/job/-/-/44412/7...,Dec 14
4,ServiceNow,Associate Machine Learning Devops Engineer Intern,"Santa Clara, CA",https://jobs.smartrecruiters.com/ServiceNow/74...,Dec 13


In [55]:
df.head()

,Company,Role,Location,Application_Link,Date_Posted
0,Zoox,Developer Platforms Intern,"Foster City, CA",https://jobs.lever.co/zoox/3021c042-c2e9-495f-...,Dec 15
1,Astranis Space Technologies,DevOps Engineer (Flight Software) - Intern 🇺🇸,"San Francisco, CA",https://job-boards.greenhouse.io/astranis/jobs...,Dec 15
2,ABB,Data Science Intern- Summer 2025,"Cary, NC",https://careers.abb/global/en/job/ABB1GLOBAL94...,Dec 15
3,Staples,Software Engineering Intern (Quill - Hybrid),"Lincolnshire, IL",https://careers.staples.com/en/job/-/-/44412/7...,Dec 14
4,ServiceNow,Associate Machine Learning Devops Engineer Intern,"Santa Clara, CA",https://jobs.smartrecruiters.com/ServiceNow/74...,Dec 13


In [58]:
enriched_jobs = enrich_job_data(df)  


Processing job 6/1590: Motorola - Data Scientist
Status: error - No text content found

Processing job 8/1590: Leidos - Data Analytics Intern
Status: error - No text content found

Processing job 10/1590: Xylem - Software Engineering Intern
Status: error - No text content found

Processing job 13/1590: Parsons - Engineering Internship
Status: error - No text content found

Processing job 14/1590: Motorola - Software Developer - Systems & Solutions
Status: error - No text content found

Processing job 7/1590: Marvell - Application Engineering Intern - Masters
Status: error - No text content found

Processing job 15/1590: Leidos - Data Scientist Intern
Status: error - No text content found

Processing job 16/1590: Intel - Undergraduate Intern Software Engineer
Status: error - No text content found

Processing job 17/1590: Docusign - Back Platform Software Engineer Intern 🛂
Status: error - Failed to fetch webpage: Status code 404

Processing job 18/1590: Vermeer - Embedded Software Inter

In [59]:
enriched_jobs.head()



,Company,Role,Location,Application_Link,Date_Posted,company_formatted,description,soft_skills,technical_skills,experience_level
0,Zoox,Developer Platforms Intern,"Foster City, CA",https://jobs.lever.co/zoox/3021c042-c2e9-495f-...,Dec 15,Zoox,The Developer Platforms Intern will assist in ...,"passionate about data, problem-solving, collab...","C++, Kotlin, Python, TypeScript, Terraform, Helm",beginner
1,Astranis Space Technologies,DevOps Engineer (Flight Software) - Intern 🇺🇸,"San Francisco, CA",https://job-boards.greenhouse.io/astranis/jobs...,Dec 15,Astranis,Astranis is seeking a DevOps Engineer Intern f...,"strong written and oral communication skills, ...","Python, git, GitHub, Bazel build system (bonus...",beginner
2,ABB,Data Science Intern- Summer 2025,"Cary, NC",https://careers.abb/global/en/job/ABB1GLOBAL94...,Dec 15,ABB,Data Science Intern position for Summer 2025 i...,"communication, teamwork, problem-solving","data analysis, machine learning, statistical m...",beginner
3,Staples,Software Engineering Intern (Quill - Hybrid),"Lincolnshire, IL",https://careers.staples.com/en/job/-/-/44412/7...,Dec 14,"Staples, Inc.",Staples is seeking a Software Engineering Inte...,"collaborative, customer-focused, inclusive, in...","C#, JavaScript, Angular, Python, Rust, SQL",beginner
4,ServiceNow,Associate Machine Learning Devops Engineer Intern,"Santa Clara, CA",https://jobs.smartrecruiters.com/ServiceNow/74...,Dec 13,ServiceNow,ServiceNow is seeking an Associate Machine Lea...,"strong written and verbal communication, creat...","Python, JavaScript, Kubernetes, Docker, REST A...",beginner


#### Filter out all jobs where page scraping wasn't succesful

In [60]:
def filter_successful_jobs(df):
    # Create a copy to avoid modifying the original DataFrame
    filtered_df = df.copy()
    
    # Filter rows where none of the enriched columns are blank
    filtered_df = filtered_df[
        (filtered_df['description'] != '') & 
        (filtered_df['soft_skills'] != '') & 
        (filtered_df['technical_skills'] != '') & 
        (filtered_df['experience_level'] != '')
    ]
    
    # Reset the index
    filtered_df = filtered_df.reset_index(drop=True)
    
    print(f"Filtered from {len(df)} to {len(filtered_df)} successfully scraped jobs")
    
    return filtered_df

successful_jobs = filter_successful_jobs(enriched_jobs)

successful_jobs.head()


Filtered from 1590 to 152 successfully scraped jobs


,Company,Role,Location,Application_Link,Date_Posted,company_formatted,description,soft_skills,technical_skills,experience_level
0,Zoox,Developer Platforms Intern,"Foster City, CA",https://jobs.lever.co/zoox/3021c042-c2e9-495f-...,Dec 15,Zoox,The Developer Platforms Intern will assist in ...,"passionate about data, problem-solving, collab...","C++, Kotlin, Python, TypeScript, Terraform, Helm",beginner
1,Astranis Space Technologies,DevOps Engineer (Flight Software) - Intern 🇺🇸,"San Francisco, CA",https://job-boards.greenhouse.io/astranis/jobs...,Dec 15,Astranis,Astranis is seeking a DevOps Engineer Intern f...,"strong written and oral communication skills, ...","Python, git, GitHub, Bazel build system (bonus...",beginner
2,ABB,Data Science Intern- Summer 2025,"Cary, NC",https://careers.abb/global/en/job/ABB1GLOBAL94...,Dec 15,ABB,Data Science Intern position for Summer 2025 i...,"communication, teamwork, problem-solving","data analysis, machine learning, statistical m...",beginner
3,Staples,Software Engineering Intern (Quill - Hybrid),"Lincolnshire, IL",https://careers.staples.com/en/job/-/-/44412/7...,Dec 14,"Staples, Inc.",Staples is seeking a Software Engineering Inte...,"collaborative, customer-focused, inclusive, in...","C#, JavaScript, Angular, Python, Rust, SQL",beginner
4,ServiceNow,Associate Machine Learning Devops Engineer Intern,"Santa Clara, CA",https://jobs.smartrecruiters.com/ServiceNow/74...,Dec 13,ServiceNow,ServiceNow is seeking an Associate Machine Lea...,"strong written and verbal communication, creat...","Python, JavaScript, Kubernetes, Docker, REST A...",beginner


#### Embed description, soft_skills, and technical_skills

In [61]:
def create_embeddings(df):
    # Create a copy of the DataFrame
    df = df.copy()
    
    # Initialize OpenAI client
    client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    
    # Function to get embedding for a text
    def get_embedding(text):
        try:
            response = client.embeddings.create(
                model="text-embedding-3-small",
                input=text,
                encoding_format="float"
            )
            return response.data[0].embedding
        except Exception as e:
            print(f"Error getting embedding: {e}")
            return None

    print("Creating embeddings...")
    
    # Create embeddings for each column
    print("Processing description embeddings...")
    df['description_embedding'] = df['description'].apply(get_embedding)
    
    print("Processing soft skills embeddings...")
    df['soft_skills_embedding'] = df['soft_skills'].apply(get_embedding)
    
    print("Processing technical skills embeddings...")
    df['technical_skills_embedding'] = df['technical_skills'].apply(get_embedding)
    
    # Check for any failed embeddings
    failed_embeddings = df[df[['description_embedding', 'soft_skills_embedding', 'technical_skills_embedding']].isna().any(axis=1)]
    if not failed_embeddings.empty:
        print(f"\nWarning: {len(failed_embeddings)} rows had failed embeddings")
        print("Failed rows:", failed_embeddings.index.tolist())
    
    print("\nEmbedding creation complete!")
    return df

In [62]:
df_with_embeddings = create_embeddings(successful_jobs)

df_with_embeddings.head()


Creating embeddings...
Processing description embeddings...
Processing soft skills embeddings...
Processing technical skills embeddings...

Embedding creation complete!


,Company,Role,Location,Application_Link,Date_Posted,company_formatted,description,soft_skills,technical_skills,experience_level,description_embedding,soft_skills_embedding,technical_skills_embedding
0,Zoox,Developer Platforms Intern,"Foster City, CA",https://jobs.lever.co/zoox/3021c042-c2e9-495f-...,Dec 15,Zoox,The Developer Platforms Intern will assist in ...,"passionate about data, problem-solving, collab...","C++, Kotlin, Python, TypeScript, Terraform, Helm",beginner,"[-0.013208664, -0.0021161127, 0.056857847, -0....","[-0.006345439, -0.01744125, 0.032430697, 0.005...","[-0.047877982, 0.03946991, 0.06380528, -0.0108..."
1,Astranis Space Technologies,DevOps Engineer (Flight Software) - Intern 🇺🇸,"San Francisco, CA",https://job-boards.greenhouse.io/astranis/jobs...,Dec 15,Astranis,Astranis is seeking a DevOps Engineer Intern f...,"strong written and oral communication skills, ...","Python, git, GitHub, Bazel build system (bonus...",beginner,"[-0.025320763, 0.054980766, 0.047757205, -0.03...","[0.020466028, 0.03162122, 0.040677663, 0.01184...","[-0.018653262, 0.062333636, 0.034288704, 0.005..."
2,ABB,Data Science Intern- Summer 2025,"Cary, NC",https://careers.abb/global/en/job/ABB1GLOBAL94...,Dec 15,ABB,Data Science Intern position for Summer 2025 i...,"communication, teamwork, problem-solving","data analysis, machine learning, statistical m...",beginner,"[0.021950189, 0.02691805, 0.05599706, 0.019941...","[0.019427849, 0.012913731, 0.049924612, 0.0219...","[0.0016983711, 0.0057976283, 0.05656223, -0.04..."
3,Staples,Software Engineering Intern (Quill - Hybrid),"Lincolnshire, IL",https://careers.staples.com/en/job/-/-/44412/7...,Dec 14,"Staples, Inc.",Staples is seeking a Software Engineering Inte...,"collaborative, customer-focused, inclusive, in...","C#, JavaScript, Angular, Python, Rust, SQL",beginner,"[-0.03976986, 0.028599696, 0.023090947, -0.019...","[0.012166299, -0.018098857, 0.034180507, 0.031...","[-0.051235348, 0.033593204, 0.022749683, 0.028..."
4,ServiceNow,Associate Machine Learning Devops Engineer Intern,"Santa Clara, CA",https://jobs.smartrecruiters.com/ServiceNow/74...,Dec 13,ServiceNow,ServiceNow is seeking an Associate Machine Lea...,"strong written and verbal communication, creat...","Python, JavaScript, Kubernetes, Docker, REST A...",beginner,"[-0.0048387363, 0.0014068669, 0.049155414, -0....","[0.03750542, 0.04007059, 0.059835866, 0.037127...","[-0.052103706, 0.011649624, 0.03565718, 0.0036..."


#### Insert jobs to Supabase

In [63]:
def insert_jobs_to_supabase(df):
    # Initialize Supabase client
    supabase_url = os.getenv('SUPABASE_URL')
    supabase_key = os.getenv('SUPABASE_ANON_KEY')
    supabase = create_client(supabase_url, supabase_key)
    
    # First, let's insert/update companies
    for index, row in df.iterrows():
        # Extract company name from markdown link if present
        company_name = row['company_formatted']
        
        # Extract company website from markdown link if present
        company_url = re.search(r'\((https?://[^\)]+)\)', row['Company'])
        company_url = company_url.group(1) if company_url else None
        
        # Insert or update company
        company_data = {
            "name": company_name,
            "website_url": company_url,
            "updated_at": datetime.now().isoformat()
        }
        
        # Upsert company (insert if not exists, update if exists)
        company_result = supabase.table('companies').upsert(company_data).execute()
        
        # Get the company id
        company_id = company_result.data[0]['id']
        
        # Handle date parsing with error checking
        date_posted = None
        if row['Date_Posted']:
            try:
                date_posted = datetime.strptime(row['Date_Posted'], '%b %d').replace(year=2023).date().isoformat()
            except ValueError:
                print(f"Warning: Could not parse date '{row['Date_Posted']}' for job {row['Role']}")
        
        # Check if job already exists
        existing_job = supabase.table('jobs')\
            .select('*')\
            .eq('company_id', company_id)\
            .eq('title', row['Role'])\
            .eq('application_url', row['Application_Link'])\
            .execute()
        
        if existing_job.data:
            print(f"Job {row['Role']} for company {company_name} already exists, skipping...")
            continue
        
        # Prepare job data including new columns and embeddings
        job_data = {
            "company_id": company_id,
            "title": row['Role'],
            "location": row['Location'],
            "application_url": row['Application_Link'],
            "date_posted": date_posted,
            "updated_at": datetime.now().isoformat(),
            
            # New enriched data columns
            "job_description": row.get('description', ''),
            "soft_skills": row.get('soft_skills', ''),
            "technical_skills": row.get('technical_skills', ''),
            "experience_level": row.get('experience_level', ''),
            
            # Vector embeddings
            "description_embedding": row.get('description_embedding', None),
            "soft_skills_embedding": row.get('soft_skills_embedding', None),
            "technical_skills_embedding": row.get('technical_skills_embedding', None)
        }
        
        # Insert job
        try:
            supabase.table('jobs').insert(job_data).execute()
            print(f"Successfully inserted job {row['Role']} for company {company_name}")
        except Exception as e:
            print(f"Error inserting job {row['Role']} for company {company_name}: {str(e)}")


In [64]:
if df_with_embeddings is not None:
    if not df_with_embeddings.empty:
        insert_jobs_to_supabase(df_with_embeddings)
        print(f"Successfully processed {len(df_with_embeddings)} jobs from today")
    else:
        print("No new jobs found today")
else:
    print("Error: Could not extract table data from README")

Successfully inserted job Developer Platforms Intern for company Zoox
Successfully inserted job DevOps Engineer (Flight Software) - Intern 🇺🇸 for company Astranis
Successfully inserted job Data Science Intern- Summer 2025 for company ABB
Successfully inserted job Software Engineering Intern (Quill - Hybrid) for company Staples, Inc.
Successfully inserted job Associate Machine Learning Devops Engineer Intern for company ServiceNow
Successfully inserted job Research Intern for company Together AI
Successfully inserted job Software Engineering Co-op - June-December 2025 for company Western Digital
Successfully inserted job Engineering Intern for company Signifyd
Successfully inserted job Summer Intern - Tech Incubation for company Samsung Research America
Successfully inserted job Wallet and Apple Pay - IOS Engineer for company Apple
Successfully inserted job Software Engineer Intern, Java Backend for company Staples, Inc.
Successfully inserted job Business Quantitative Analyst Internship